Thanks to user Dima for giving context of variables in his code, since there is no data dictionary available. Some part of his code were used for data cleaning since they were optimal.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

### import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
 
from sklearn.model_selection import train_test_split, ShuffleSplit, learning_curve, GridSearchCV, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import roc_curve, accuracy_score, confusion_matrix, classification_report, roc_auc_score, make_scorer, precision_recall_curve, average_precision_score 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier, IsolationForest, VotingClassifier
from sklearn.neural_network import MLPClassifier

%matplotlib inline
plt.style.use('ggplot')

from catboost import CatBoostClassifier # Or CatBoostRegressor
from sklearn.model_selection import KFold
from itertools import product,chain

import shap
import catboost
from catboost import *
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import recall_score, accuracy_score, confusion_matrix, classification_report
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


np.random.seed(0)
random.seed(0)
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sortedcontainers import SortedList
import copy
import collections
import numpy as np
from itertools import product,chain
import pandas
from sklearn.model_selection import KFold
import catboost as cb

''' a class for doing grid search on a set of parameters provided in a dict. 'pdict' should be a dictionary like the following:
pdict = {'depth':[1,2], 'iterations':[250,100,500], 'thread_count':4}

when grid_search is called it will return an iterator that provides samples from the dictionary e.g.
{'depth':1, 'iterations':250, 'thread_count':4}
{'depth':2, 'iterations':250, 'thread_count':4}
{'depth':1, 'iterations':100, 'thread_count':4}
etc.
after calling an iteration of grid_search, you need to test the classifier and run 'register_result'
This will update the internal list of results, so that the next call to grid_search will use the best
parameters for all the parameters not currently being updated.

grid_search can be provided a list e.g. grid_search(['depth']) this will use the current best parameters for all
the other arguments and only search over 'depth'. You can then call e.g. grid_search(['iterations']) and it will use
the best depth found previously and cycle through all the 'iterations'. Searching incrementally can be much faster
than doing a full grid search, but may miss the global optimum. '''
class paramsearch:
    def __init__(self,pdict):    
        self.pdict = {}
        # if something is not passed in as a sequence, make it a sequence with 1 element
        #   don't treat strings as sequences
        for a,b in pdict.items():
            if isinstance(b, collections.Sequence) and not isinstance(b, str): self.pdict[a] = b
            else: self.pdict[a] = [b]
        # our results are a sorted list, so the best score is always the final element
        self.results = SortedList()       
                    
    def grid_search(self,keys=None):
        # do grid search on only the keys listed. If none provided, do all
        if keys==None: keylist = self.pdict.keys()
        else: keylist = keys
 
        listoflists = [] # this will be list of lists of key,value pairs
        for key in keylist: listoflists.append([(key,i) for i in self.pdict[key]])
        for p in product(*listoflists):
            # do any changes to the current best parameter set
            if len(self.results)>0: template = self.results[-1][1]
            else: template = {a:b[0] for a,b in self.pdict.items()}
            # if our updates are the same as current best, don't bother
            if self.equaldict(dict(p),template): continue
            # take the current best and update just the ones to change
            yield self.overwritedict(dict(p),template)
                              
    def equaldict(self,a,b):
        for key in a.keys(): 
            if a[key] != b[key]: return False
        return True            
                              
    def overwritedict(self,new,old):
        old = copy.deepcopy(old)
        for key in new.keys(): old[key] = new[key]
        return old            
    
    # save a (score,params) pair to results. Since 'results' is a sorted list,
    #   the best score is always the final element. A small amount of noise is added
    #   because sorted lists don't like it when two scores are exactly the same    
    def register_result(self,result,params):
        self.results.add((result+np.random.randn()*1e-10,params))    
        
    def bestscore(self):
        return self.results[-1][0]
        
    def bestparam(self):
        return self.results[-1][1]
        

In [ ]:
# Function to load the inputs dataset
def read_data(tp = "Train", N = 1542865627584):
    target = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/{}-{}.csv".format(tp.title(), N))
    pt = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/{}_Beneficiarydata-{}.csv".format(tp.title(), N))
    in_pt = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/{}_Inpatientdata-{}.csv".format(tp.title(), N))
    out_pt = pd.read_csv("/kaggle/input/healthcare-provider-fraud-detection-analysis/{}_Outpatientdata-{}.csv".format(tp.title(), N))
    return (in_pt, out_pt, pt, target)

In [ ]:
# This function helps us find the unique values in a row
def N_unique_values(df):
    return np.array([len(set([i for i in x[~pd.isnull(x)]])) for x in df.values])

In [ ]:
### Load Train data
in_pt, out_pt, ben, target = read_data()

# Data Cleaning

In [ ]:
# Replace values with a binary annotation
ben = ben.replace({'ChronicCond_Alzheimer': 2, 'ChronicCond_Heartfailure': 2, 'ChronicCond_KidneyDisease': 2,
                   'ChronicCond_Cancer': 2, 'ChronicCond_ObstrPulmonary': 2, 'ChronicCond_Depression': 2,
                   'ChronicCond_Diabetes': 2, 'ChronicCond_IschemicHeart': 2, 'ChronicCond_Osteoporasis': 2,
                   'ChronicCond_rheumatoidarthritis': 2, 'ChronicCond_stroke': 2, 'Gender': 2 }, 
                  0)
ben = ben.replace({'RenalDiseaseIndicator': 'Y'}, 1).astype({'RenalDiseaseIndicator': 'int64'})

# Change target variable to binary
target["target"] = np.where(target.PotentialFraud == "Yes", 1, 0) 
target.drop('PotentialFraud', axis=1, inplace=True)

In [ ]:
# Merge in_pt, out_pt and ben df into a single patient dataset
data = pd.merge(in_pt, out_pt,
                    left_on = [ idx for idx in out_pt.columns if idx in in_pt.columns],
                    right_on = [ idx for idx in out_pt.columns if idx in in_pt.columns],
                    how = 'outer').\
          merge(ben,left_on='BeneID',right_on='BeneID',how='inner')

In [ ]:
patient_merge_id = [i for i in out_pt.columns if i in in_pt.columns]

# Merge in_pt, out_pt and ben df into a single patient dataset
data = pd.merge(in_pt, out_pt,
                    left_on = patient_merge_id,
                    right_on = patient_merge_id,
                    how = 'outer').\
          merge(ben,left_on='BeneID',right_on='BeneID',how='inner')


# Feature engineering 

In [ ]:
# We find the number of unique physicians 
data['N_unique_Physicians'] = N_unique_values(data[['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']]) 

# We separate the types of physicians into numeric values
data[['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']] = np.where(data[['AttendingPhysician','OperatingPhysician',
                                                                                      'OtherPhysician']].isnull(), 0, 1)

# We count the number of types of physicians that attend the patient
data['N_Types_Physicians'] = data['AttendingPhysician'] +  data['OperatingPhysician'] + data['OtherPhysician']

# Now we create a variable to check if there is a single doctor on a patient that was attended by more than 1 type of doctor
# This helps us finds those cases that are only looked at by 1 physicians
data['Same_Physician'] = data.apply(lambda x: 1 if (x['N_unique_Physicians'] == 1 and x['N_Types_Physicians'] > 1) else 0,axis=1)

# Similar to Same_Physician, we create a variable to see if 1 physicians has had multiple roles, but has not been alone reviewing the case
data['Same_Physician2'] = data.apply(lambda x: 1 if (x['N_unique_Physicians'] == 2 and x['N_Types_Physicians'] > 2) else 0,axis=1)

# We check our new variables
data[['N_unique_Physicians','N_Types_Physicians','Same_Physician','Same_Physician2']].head()

In [ ]:
# We count the number of procedures for each claim, we drop the initial variables
ClmProcedure_vars = ['ClmProcedureCode_{}'.format(x) for x in range(1,7)]
data['N_Procedure'] = N_unique_values(data[ClmProcedure_vars])
data = data.drop(ClmProcedure_vars, axis = 1)

# We count the number of claims, we also separate this by unique claims and extra claims, we drop the initial variables
ClmDiagnosisCode_vars =['ClmAdmitDiagnosisCode'] + ['ClmDiagnosisCode_{}'.format(x) for x in range(1, 11)]

data['N_Unique_Claims'] = N_unique_values(data[ClmDiagnosisCode_vars])
data['N_Total_Claims'] = data[ClmDiagnosisCode_vars].notnull().to_numpy().sum(axis = 1)
data['N_Extra_Claims'] = data['N_Total_Claims'] - data['N_Unique_Claims']

ClmDiagnosisCode_vars.append('N_Total_Claims')
data = data.drop(ClmDiagnosisCode_vars, axis = 1)

In [ ]:
#  Transform string columns of date into type date
data['AdmissionDt'] = pd.to_datetime(data['AdmissionDt'] , format = '%Y-%m-%d')
data['DischargeDt'] = pd.to_datetime(data['DischargeDt'],format = '%Y-%m-%d')

data['ClaimStartDt'] = pd.to_datetime(data['ClaimStartDt'] , format = '%Y-%m-%d')
data['ClaimEndDt'] = pd.to_datetime(data['ClaimEndDt'],format = '%Y-%m-%d')

data['DOB'] = pd.to_datetime(data['DOB'] , format = '%Y-%m-%d')
data['DOD'] = pd.to_datetime(data['DOD'],format = '%Y-%m-%d')

# Number of days
data['Admission_Days'] = ((data['DischargeDt'] - data['AdmissionDt']).dt.days) + 1

# Number of claim days 
data['Claim_Days'] = ((data['ClaimEndDt'] - data['ClaimStartDt']).dt.days) + 1

# Age at the time of claim
data['Age'] = round(((data['ClaimStartDt'] - data['DOB']).dt.days + 1)/365.25)

In [ ]:
# We create a Hospitalization flag 
data['Hospt'] = np.where(data.DiagnosisGroupCode.notnull(), 1, 0)
data = data.drop(['DiagnosisGroupCode'], axis = 1)

# Variable if patient is dead
data['Dead']= 0
data.loc[data.DOD.notna(),'Dead'] = 1

# Handling missing data

In [ ]:
# We find which variables hold missing data
na = data.isnull().sum()
na[na != 0]

In [ ]:
## We know that missing admission days come from missing admission and discharge date, and those cases come from the out patients data set


# We also see that there are some cases of missing deductible amount paid, so we also want to keep an eye on that
data['Missing_Deductible_Amount_Paid'] = 0
data.loc[data['DeductibleAmtPaid'].isnull(), 'Missing_Deductible_Amount_Paid'] = 1 

# After identifying the missing values, we fill the missing values with 0
data = data.fillna(0).copy()

# Create final Datasets

In [ ]:
### Sum all numeric variables
_sum = data.groupby(['Provider'], as_index = False)[['InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'RenalDiseaseIndicator', 
                                                    'ChronicCond_Alzheimer', 'AttendingPhysician', 'OperatingPhysician', 
                                                    'OtherPhysician', 'N_unique_Physicians', 'ChronicCond_Heartfailure', 
                                                    'N_Types_Physicians', 'Same_Physician',
                                                    'ChronicCond_KidneyDisease', 'ChronicCond_Cancer', 
                                                    'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression', 
                                                    'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart', 
                                                    'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
                                                    'ChronicCond_stroke', 'Dead', 
                                                    'N_Procedure','N_Unique_Claims', 'N_Extra_Claims', 'Admission_Days',
                                                    'Claim_Days', 'Hospt', 'Missing_Deductible_Amount_Paid']].sum()

# To separate our variables, we shall add '_sum' at the end of their names
_sum = _sum.add_suffix('_sum')


### Count number of records
_count = data[['BeneID', 'ClaimID']].groupby(data['Provider']).nunique().reset_index()
_count.rename(columns={'BeneID':'BeneID_count','ClaimID':'ClaimID_count'},inplace=True)


### Calculate mean for all numeric variables
_mean = data.groupby(['Provider'], as_index = False)[['NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
                                                      'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
                                                      'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt', 'Age',
                                                      'AttendingPhysician', 'OperatingPhysician','OtherPhysician',
                                                      'N_unique_Physicians', 'ChronicCond_Heartfailure', 
                                                      'N_Types_Physicians', 'Same_Physician',
                                                      'ChronicCond_KidneyDisease', 'ChronicCond_Cancer', 
                                                      'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression', 
                                                      'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart', 
                                                      'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
                                                      'ChronicCond_stroke', 'Dead', 'N_Procedure','N_Unique_Claims', 
                                                      'N_Extra_Claims', 'Admission_Days','Claim_Days', 'Hospt',
                                                      'Missing_Deductible_Amount_Paid'
                                                   ]].mean()

# To separate our variables, we shall add '_mean' at the end of their names
_mean = _mean.add_suffix('_mean')

# We create a dataset that holds all the variables
_total = _count.merge(_sum, how='left',left_on='Provider',right_on='Provider_sum').\
                merge(_mean, how='left',left_on='Provider',right_on='Provider_mean').\
                drop(['Provider_sum','Provider_mean'], axis=1).\
                merge(target, on='Provider', how='left')

In [ ]:
_total.groupby( ["target"] ).target.count().plot(kind = "bar", figsize = (10,6));

# Feature Selection - Correlation

In [ ]:
def plot_corr(df_corr):
    corrMatrix = df_corr.corr()
    plt.subplots(figsize=(20,15))
    sns.heatmap(corrMatrix, annot=False)
    plt.show()

In [ ]:
# We plot the correlations in the _sum dataset
plot_corr(_sum)

In [ ]:
# We drop correlated values 
sum_corr = _sum.drop(['Hospt_sum','AttendingPhysician_sum','OperatingPhysician_sum','Admission_Days_sum',
                      'OtherPhysician_sum','ChronicCond_ObstrPulmonary_sum', 'ChronicCond_Depression_sum',
                      'ChronicCond_Diabetes_sum','ChronicCond_IschemicHeart_sum','ChronicCond_KidneyDisease_sum', 
                      'ChronicCond_Cancer_sum','ChronicCond_Osteoporasis_sum','RenalDiseaseIndicator_sum',
                      'ChronicCond_rheumatoidarthritis_sum','ChronicCond_Heartfailure_sum','N_Unique_Claims_sum',
                      'ChronicCond_Alzheimer_sum','ChronicCond_stroke_sum','N_Procedure_sum','N_unique_Physicians_sum',
                      'N_Types_Physicians_sum','DeductibleAmtPaid_sum'],axis=1) 

# And plot again to check
plot_corr(sum_corr)

In [ ]:
# We plot the correlations in the _mean dataset
plot_corr(_mean)

In [ ]:
# We drop correlated values 
mean_corr = _mean.drop(['Hospt_mean','Admission_Days_mean','N_unique_Physicians_mean','NoOfMonths_PartBCov_mean',
                        'NoOfMonths_PartACov_mean','IPAnnualReimbursementAmt_mean','N_Procedure_mean',
                        'OPAnnualDeductibleAmt_mean', 'N_Unique_Claims_mean'], axis=1) 

# And plot again to check
plot_corr(mean_corr)

# Feature Selection - SHAP values
    We shall use shap plots from a catboost model to find the best parameters

In [ ]:
# this function does 3-fold crossvalidation with catboostclassifier          
def cross_val_test(params,train_set,train_label,cat_dims,n_splits=3):
    Skf = StratifiedKFold(n_splits=n_splits,shuffle=True, random_state = 0) 
    res = []
    for train_index, test_index in Skf.split(train_set,train_label):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]
        
        labels = train_label.iloc[train_index]
        test_labels = train_label.iloc[test_index]

        clf = CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims,verbose=False)

        # We use the recall score to search for the best parameters
        res.append(recall_score(test_labels, clf.predict(test)))
    return np.mean(res)

In [ ]:
# this function runs grid search on several parameters
def catboost_param_tune(params,train_set,train_label,cat_dims=None,n_splits=3):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(ps.grid_search(['border_count']),
                      ps.grid_search(['l2_leaf_reg']),
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['depth'])):
        res = cross_val_test(prms,train_set,train_label,cat_dims,n_splits)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        print(ps.bestscore())
    return ps.bestparam()


In [ ]:
# This function runs a catboost model on the data we define
def fcatboost(train, test):
    
    X_train, X_test, y_train, y_test = train_test_split(train, test, test_size=0.30, random_state=1)
    
    # convert categorical columns to integers
    category_cols = []
    cat_dims = [train_set.columns.get_loc(i) for i in category_cols[:-1]] 
    
    column_names = X_train.columns
    
    X_train = pd.DataFrame(StandardScaler().fit_transform(X_train))
    X_test = pd.DataFrame(StandardScaler().fit_transform(X_test))
    
    X_train.columns = column_names
    X_test.columns = column_names

    params = {'depth':[3,1,2,6,4,5,7,8,9,10],
              'iterations':[250,100,500,1000],
              'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
              'l2_leaf_reg':[3,1,5,10,100],
              'border_count':[32,5,10,20,50,100,200],
              'thread_count':4,
              'random_seed':2}

    bestparams = catboost_param_tune(params,X_train,y_train,[])

    model = CatBoostClassifier(**bestparams)
    model.fit(X_train,y_train,verbose=False)
    print(model.random_seed_)
    
    return model, X_train, X_test, y_train, y_test

In [ ]:
# This function plots the shap values from the model
def catboost_shap_plot(model, X_train, y_train):
    
    shap.initjs()

    shap_values = model.get_feature_importance(Pool(X_train,y_train), type='ShapValues')

    expected_value = shap_values[0,-1]
    shap_values = shap_values[:,:-1]
    print('\nShap features importance:')

    # summarize the effects of all the features
    shap.summary_plot(shap_values, X_train, plot_type="bar")
    shap.summary_plot(shap_values, X_train)
    

We run the catboost model on the sum dataset first

In [ ]:


sum_corr = sum_corr.merge(target, left_on='Provider_sum', right_on='Provider', how='left')

model, X_train, X_test, y_train, y_test = fcatboost(sum_corr.drop(['Provider','target','Provider_sum'], axis = 1),
                                                    sum_corr.target)

In [ ]:
catboost_shap_plot(model, X_train, y_train)

>     With this graph, we decide on the variables InscClaimAmtReimbursed_sum and Claim_Days_sum


We run the catboost model on the mean dataset 

In [ ]:
mean_corr = mean_corr.merge(target, left_on='Provider_mean', right_on='Provider', how='left')

model, X_train, X_test, y_train, y_test = fcatboost(mean_corr.drop(['Provider','target','Provider_mean'], axis = 1),
                                                    mean_corr.target)

In [ ]:
catboost_shap_plot(model, X_train, y_train)

    Looking at the graph of SHAP values, we decide to use the variables: AttendingPhysician_mean,Missing_Deductible_Amount_Paid_mean,Dead_mean,Claim_Days_mean and N_Extra_Claims_mean

# Features Selection - Final draft

Based on the previous shap plots we can obtain the variables of interest from the sum and mean datasets to test our models, we also add the variables from the count dataset and the provider and target.

In [ ]:
df = _total[['InscClaimAmtReimbursed_sum','N_Extra_Claims_sum','Claim_Days_sum',
             'AttendingPhysician_mean','Missing_Deductible_Amount_Paid_mean','Dead_mean','Claim_Days_mean','N_Extra_Claims_mean',
             'BeneID_count','ClaimID_count',
             'Provider','target']]

 # Modeling - Catboost

In [ ]:
def cm_Score(model, df):
    recall = np.mean(cross_val_score(model, df.drop(['Provider','target'], axis = 1), df.target, cv=3, scoring='recall'))
    accuracy = np.mean(cross_val_score(model, df.drop(['Provider','target'], axis = 1), df.target, cv=3, scoring='accuracy'))
    
    print('Accuracy Score: {}'.format(accuracy))
    print('Recall Score: {}'.format(recall))
   

In [ ]:
model, X_train, X_test, y_train, y_test = fcatboost(df.drop(['Provider','target'], axis = 1), df.target)

    The best catboost model has recall of 0.53

# Modeling - Logistic Regression

In [ ]:
def flogistic(df, penalty):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(['Provider','target'], axis = 1),df.target, 
                                                  test_size=0.30, random_state=1)

    X_train = pd.DataFrame(StandardScaler().fit_transform(X_train))
    X_test = pd.DataFrame(StandardScaler().fit_transform(X_test))
    
    #Liblinear for small datasets of binary classes
    logreg = LogisticRegression(penalty= penalty,solver= 'liblinear',class_weight='balanced', random_state = 5 , C = 0.001)
    
    cm_Score(logreg, df)
    
    

In [ ]:
flogistic(df, 'l1')

acc .8
rec 0.91

In [ ]:
flogistic(df, 'l2')

acc .58
rec 0.97

# Modeling - Naive Bayes

In [ ]:
def Gaus_bayes(df):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(['Provider','target'], axis = 1),df.target, 
                                                  test_size=0.30, random_state=1)


    X_train = pd.DataFrame(StandardScaler().fit_transform(X_train))
    X_test = pd.DataFrame(StandardScaler().fit_transform(X_test))

    gnb = GaussianNB()
    
    cm_Score(gnb, df)

In [ ]:
Gaus_bayes(df)

    The naive bayes model has a recall of 0.50